# Import packages

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import xgboost as xgb
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics import precision_score, recall_score

# Define parameters and hyperparameters

## BPIC 2017

In [ ]:
suffix_len = 88
num_act = 29
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Trace-based model

In [ ]:
# hyperparameters for training an XGBoost model to predict the next activity label
param_act = {
        'objective': 'multi:softprob',
        'eval_metric': 'mlogloss',
        'num_class': num_act,
        'max_depth': 8,
        'min_child_weight': 9.74462727911892,
        'subsample': 0.9618841777880232,
        'colsample_bytree': 0.6582241534026132,
        'lambda': 8.002051385874758,
        'eta': 0.10167697633502167
    }

# hyperparameters for training an XGBoost model to predict the next timestamp
param_time = {
        'objective': 'reg:absoluteerror',
        'eval_metric': 'mae',
        'max_depth': 12,
        'min_child_weight': 12.337223244031959,
        'subsample': 0.8453782420681207,
        'colsample_bytree': 0.9033521246902182,
        'lambda': 8.3746228993637,
        'eta': 0.2196528959499405
    }

### Integrated model

In [ ]:
# hyperparameters for training an XGBoost model to predict the next activity label
param_act = {
        'objective': 'multi:softprob',
        'eval_metric': 'mlogloss',
        'num_class': num_act,
        'max_depth': 10,
        'min_child_weight': 4.261024311224565,
        'subsample': 0.9804376496746907,
        'colsample_bytree': 0.9187764362444105,
        'lambda': 7.912072508972737,
        'eta': 0.10394052247291184
    }

# hyperparameters for training an XGBoost model to predict the next timestamp
param_time = {
        'objective': 'reg:absoluteerror',
        'eval_metric': 'mae',
        'max_depth': 11,
        'min_child_weight': 11.667738097521406,
        'subsample': 0.9240363624981516,
        'colsample_bytree': 0.9785011357939114,
        'lambda': 3.1663267935061397,
        'eta': 0.11949621033263035
    }

## BPIC 2019

In [ ]:
suffix_len = 14
num_act = 40
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Trace-based model

In [ ]:
# hyperparameters for training an XGBoost model to predict the next activity label
param_act = {
        'objective': 'multi:softprob',
        'eval_metric': 'mlogloss',
        'num_class': num_act,
        'max_depth': 7,
        'min_child_weight': 2.8049924942971702,
        'subsample': 0.6638855846724679,
        'colsample_bytree': 0.7864021562769484,
        'lambda': 7.926181642148752,
        'eta': 0.14555480478649802
    }

# hyperparameters for training an XGBoost model to predict the next timestamp
param_time = {
        'objective': 'reg:absoluteerror',
        'eval_metric': 'mae',
        'max_depth': 12,
        'min_child_weight': 17.011718425833887,
        'subsample': 0.8548789057622406,
        'colsample_bytree': 0.6225615621294496,
        'lambda': 3.8661288379363032,
        'eta': 0.2469307468511154
    }

### Integrated model

In [ ]:
# hyperparameters for training an XGBoost model to predict the next activity label
param_act = {
        'objective': 'multi:softprob',
        'eval_metric': 'mlogloss',
        'num_class': num_act,
        'max_depth': 10,
        'min_child_weight': 4.261024311224565,
        'subsample': 0.9804376496746907,
        'colsample_bytree': 0.9187764362444105,
        'lambda': 7.912072508972737,
        'eta': 0.10394052247291184
    }

# hyperparameters for training an XGBoost model to predict the next timestamp
param_time = {
        'objective': 'reg:absoluteerror',
        'eval_metric': 'mae',
        'max_depth': 12,
        'min_child_weight': 8.53429129870081,
        'subsample': 0.7912897584098408,
        'colsample_bytree': 0.9122487127320722,
        'lambda': 3.4505487528134626,
        'eta': 0.14425392950846272
    }

# Define file path

In [ ]:
# define file path
train_log_prefix_tensor_path = '.../train_log_prefix_tensor.pt'
train_trace_prefix_tensor_path = '.../train_trace_prefix_tensor.pt'
train_suffix_act_tensor_path = '.../train_suffix_act_tensor.pt'
train_suffix_time_tensor_path = '.../train_suffix_time_tensor.pt'

val_log_prefix_tensor_path = '.../val_log_prefix_tensor.pt'
val_trace_prefix_tensor_path = '.../val_trace_prefix_tensor.pt'
val_suffix_act_tensor_path = '.../val_suffix_act_tensor.pt'
val_suffix_time_tensor_path = '.../val_suffix_time_tensor.pt'

test_log_prefix_tensor_path = '.../test_log_prefix_tensor.pt'
test_trace_prefix_tensor_path = '.../test_trace_prefix_tensor.pt'
test_suffix_act_tensor_path = '.../test_suffix_act_tensor.pt'
test_suffix_time_tensor_path = '.../test_suffix_time_tensor.pt'

In [ ]:
# get min and max values
max_dict = torch.load('.../max_dict.pt')
min_dict = torch.load('.../min_dict.pt')

min_pre = min_dict['trace_ts_pre']
max_pre = max_dict['trace_ts_pre']

min_start = min_dict['trace_ts_start']
max_start = max_dict['trace_ts_start']

# Prepare feature vector

## Trace-based model

In [ ]:
# X_train
train_trace_prefix_tensor = torch.load(train_trace_prefix_tensor_path) # shape: (num_samples, trace_prefix_len, num_act + 2)
train_trace_prefix_tensor = train_trace_prefix_tensor.reshape(train_trace_prefix_tensor.shape[0], -1) # shape: (num_samples, trace_prefix_len * (num_act + 2))
X_train = train_trace_prefix_tensor.numpy()

In [ ]:
# X_val
val_trace_prefix_tensor = torch.load(val_trace_prefix_tensor_path) # shape: (num_samples, trace_prefix_len, num_act + 2)
val_trace_prefix_tensor = val_trace_prefix_tensor.reshape(val_trace_prefix_tensor.shape[0], -1) # shape: (num_samples, trace_prefix_len * (num_act + 2))
X_val = val_trace_prefix_tensor.numpy()

In [ ]:
# X_test
test_trace_prefix_tensor = torch.load(test_trace_prefix_tensor_path) # shape: (num_samples, trace_prefix_len, num_act + 2)
test_trace_prefix_tensor = test_trace_prefix_tensor.reshape(test_trace_prefix_tensor.shape[0], -1) # shape: (num_samples, trace_prefix_len * (num_act + 2))
X_test = test_trace_prefix_tensor.numpy()

## Integrated model

In [ ]:
# X_train
train_log_prefix_tensor = torch.load(train_log_prefix_tensor_path) # shape: (num_samples, log_prefix_len, num_act + 1)
train_log_prefix_tensor = train_log_prefix_tensor.reshape(train_log_prefix_tensor.shape[0], -1) # shape: (num_samples, (log_prefix_len * (num_act + 1))
# remove the activity label of the last event in the log prefix, 
# as it is redundant (already captured in the trace prefix).
train_log_prefix_tensor = torch.cat(
    (train_log_prefix_tensor[:, :-(num_act+1)], train_log_prefix_tensor[:, -1].unsqueeze(1)),
    dim=1)

train_trace_prefix_tensor = torch.load(train_trace_prefix_tensor_path) # shape: (num_samples, trace_prefix_len, num_act + 2)
train_trace_prefix_tensor = train_trace_prefix_tensor.reshape(train_trace_prefix_tensor.shape[0], -1) # shape: (num_samples, trace_prefix_len * (num_act + 2))

X_train_tensor = torch.cat((train_log_prefix_tensor, train_trace_prefix_tensor), -1)
X_train = X_train_tensor.numpy()

In [ ]:
# X_val
val_log_prefix_tensor = torch.load(val_log_prefix_tensor_path) # shape: (num_samples, log_prefix_len, num_act + 1)
val_log_prefix_tensor = val_log_prefix_tensor.reshape(val_log_prefix_tensor.shape[0], -1) # shape: (num_samples, (log_prefix_len * (num_act + 1))
# remove the activity label of the last event in the log prefix, 
# as it is redundant (already captured in the trace prefix).
val_log_prefix_tensor = torch.cat(
    (val_log_prefix_tensor[:, :-(num_act+1)], val_log_prefix_tensor[:, -1].unsqueeze(1)),
    dim=1)

val_trace_prefix_tensor = torch.load(val_trace_prefix_tensor_path) # shape: (num_samples, trace_prefix_len, num_act + 2)
val_trace_prefix_tensor = val_trace_prefix_tensor.reshape(val_trace_prefix_tensor.shape[0], -1) # shape: (num_samples, trace_prefix_len * (num_act + 2))

X_val_tensor = torch.cat((val_log_prefix_tensor, val_trace_prefix_tensor), -1)
X_val = X_val_tensor.numpy()

In [ ]:
# X_test
test_log_prefix_tensor = torch.load(test_log_prefix_tensor_path) # shape: (num_samples, log_prefix_len, num_act + 1)
test_log_prefix_tensor = test_log_prefix_tensor.reshape(test_log_prefix_tensor.shape[0], -1) # shape: (num_samples, (log_prefix_len * (num_act + 1))
# remove the activity label of the last event in the log prefix, 
# as it is redundant (already captured in the trace prefix).
test_log_prefix_tensor = torch.cat(
    (test_log_prefix_tensor[:, :-(num_act+1)], test_log_prefix_tensor[:, -1].unsqueeze(1)),
    dim=1)

test_trace_prefix_tensor = torch.load(test_trace_prefix_tensor_path) # shape: (num_samples, trace_prefix_len, num_act + 2)
test_trace_prefix_tensor = test_trace_prefix_tensor.reshape(test_trace_prefix_tensor.shape[0], -1) # shape: (num_samples, trace_prefix_len * (num_act + 2))

X_test_tensor = torch.cat((test_log_prefix_tensor, test_trace_prefix_tensor), -1)
X_test = X_test_tensor.numpy()

# Prepare target

In [ ]:
# Next activity label

# y_train_act
train_suffix_act_tensor = torch.load(train_suffix_act_tensor_path)
y_train_act_tensor = train_suffix_act_tensor[:, 0]
y_train_act = y_train_act_tensor.numpy()

# y_val_act
val_suffix_act_tensor = torch.load(val_suffix_act_tensor_path)
y_val_act_tensor = val_suffix_act_tensor[:, 0]
y_val_act = y_val_act_tensor.numpy()

# y_test_act
test_suffix_act_tensor = torch.load(test_suffix_act_tensor_path)
y_test_act_tensor = test_suffix_act_tensor[:, 0]
y_test_act = y_test_act_tensor.numpy()

In [ ]:
# y_train_time
train_suffix_time_tensor = torch.load(train_suffix_time_tensor_path)
y_train_time_tensor = train_suffix_time_tensor[:, 0]
y_train_time = y_train_time_tensor.numpy()

# y_val_time
val_suffix_time_tensor = torch.load(val_suffix_time_tensor_path)
y_val_time_tensor = val_suffix_time_tensor[:, 0]
y_val_time = y_val_time_tensor.numpy()

# y_test_time
test_suffix_time_tensor = torch.load(test_suffix_time_tensor_path)
y_test_time_tensor = test_suffix_time_tensor[:, 0]
y_test_time = y_test_time_tensor.numpy()

# XGBoost for next activity label prediction

In [ ]:
# train XGBoost
dtrain_act = xgb.DMatrix(data=X_train, label=y_train_act)
dvalid_act = xgb.DMatrix(data=X_val, label=y_val_act)

model_act = xgb.train(param_act, 
                   dtrain_act, 
                   evals=[(dtrain_act, 'train'), (dvalid_act, 'validation')], 
                   num_boost_round = 100000,
                   early_stopping_rounds = 20,
                   verbose_eval = 20)

In [ ]:
# evaluate next activity label prediction performance on the test set
dtest = xgb.DMatrix(X_test)
y_pred_act = model_act.predict(dtest)
y_labels = np.argmax(y_pred_act, axis=1)

precision = precision_score(y_test_act, y_labels, average='weighted')
recall = recall_score(y_test_act, y_labels, average='weighted')
print(precision)
print(recall)

# XGBoost for next timestamp prediction

In [ ]:
# train XGBoost
dtrain_time = xgb.DMatrix(data=X_train, label=y_train_time)
dvalid_time = xgb.DMatrix(data=X_val, label=y_val_time)

model_time= xgb.train(param_time, 
                   dtrain_time, 
                   evals=[(dtrain_time, 'train'), (dvalid_time, 'validation')], 
                   num_boost_round = 100000,
                   early_stopping_rounds = 20,
                   verbose_eval = 20)

In [ ]:
# evaluate next timestamp prediction performance on the test set
dtest = xgb.DMatrix(X_test)
y_pred_time = model_time.predict(dtest)

y_pred_time_norm = y_pred_time* (max_pre - min_pre) + min_pre # reverse max-min normalization
y_pred_time_log = np.expm1(y_pred_time_norm) # reverse log-transformation

y_test_time_norm= y_test_time* (max_pre - min_pre) + min_pre # reverse max-min normalization
y_test_time_log = np.expm1(y_test_time_norm) # reverse log-transformation

mae = mean_absolute_error(y_test_time_log, y_pred_time_log)
msle = mean_squared_error(y_test_time_norm, y_pred_time_norm)
print(mae)
print(msle)

# Recursively generate suffix

In [ ]:
act_predictions = np.zeros((X_test.shape[0], suffix_len, num_act))
time_predictions = np.zeros((X_test.shape[0], suffix_len, 1))

X_test_renew = X_test.copy()

for i in range(suffix_len):

    dtest = xgb.DMatrix(X_test_renew)

    # -- predict TTNE -- #

    y_time_pre = model_time.predict(dtest) # shape: (val_size, )
    y_time_pre_2d = y_time_pre.reshape(-1, 1) # shape: (val_size, 1)
    
    # -- derive time since case starts -- #

    # get old normalized y_time_start
    old_y_time_start = X_test_renew[:, -2] # shape: (val_size, )

    # denormalize y_time_start
    old_y_time_start = old_y_time_start* (max_start - min_start) + min_start
    old_y_time_start = np.expm1(old_y_time_start)

    # denormalize predicted TTNE
    y_time_pre_denorm = y_time_pre* (max_pre - min_pre) + min_pre
    y_time_pre_denorm = np.expm1(y_time_pre_denorm)

    # add old_y_time_start and predicted TTNE to get new y_time_start
    y_time_start = old_y_time_start + y_time_pre_denorm

    # log normalize y_time_start
    y_time_start = np.maximum(y_time_start, 0) 
    y_time_start = np.log1p(y_time_start)
    y_time_start = (y_time_start - min_start) / (max_start - min_start)

    # -- predict activity label --
    y_act = model_act.predict(dtest) # shape: (val_size, num_act)
    y_labels = np.argmax(y_act, axis=1) # shape: (val_size,)
    y_act_hot = np.eye(num_act)[y_labels] # shape: (val_size, num_act)
    y_act_hot[:, 0] = 0 

    act_predictions[:, i, :] = y_act
    time_predictions[:, i, :] = y_time_pre_2d

    # update X_test
    X_test_renew[:, -((num_act+2) * suffix_len): -(num_act+2)] = X_test_renew[:, -((num_act+2) * (suffix_len-1)):]
    X_test_renew[:, -(num_act+2):-2] = y_act_hot
    X_test_renew[:, -2] = y_time_start
    X_test_renew[:, -1] = y_time_pre

act_predictions = torch.from_numpy(act_predictions)
time_predictions = torch.from_numpy(time_predictions)

In [ ]:
def damerau_levenshtein(list1, 
                        list2):
    """
    Compute the Damerau-Levenshtein distance between two sequences of activity labels.

    The Damerau-Levenshtein distance measures the minimum number of operations 
    (insertions, deletions, substitutions, and transpositions of adjacent elements) 
    required to transform one list into the other.

    Parameters
    ----------
    list1 : list
        First sequence of activity labels.
    list2 : list
        Second sequence of activity labels.

    Returns
    -------
    dl_distance : float
        The computed Damerau-Levenshtein distance between the two sequences.
    """
    len_1, len_2 = len(list1), len(list2)

    dist = [[0 for _ in range(len_2 + 1)] for _ in range(len_1 + 1)]

    for i in range(len_1 + 1):
        dist[i][0] = i
    for j in range(len_2 + 1):
        dist[0][j] = j

    for i in range(1, len_1 + 1):
        for j in range(1, len_2 + 1):
            cost = 0 if list1[i - 1] == list2[j - 1] else 1

            dist[i][j] = min(
                dist[i - 1][j] + 1,    # deletion
                dist[i][j - 1] + 1,    # insertion
                dist[i - 1][j - 1] + cost  # substitution
            )

            if i > 1 and j > 1 and list1[i - 1] == list2[j - 2] and list1[i - 2] == list2[j - 1]:
                dist[i][j] = min(
                    dist[i][j],
                    dist[i - 2][j - 2] + cost  # transposition
                )

    dl_distance = dist[len_1][len_2]

    return dl_distance

def performance_metrics(act_predictions, 
                        time_predictions, 
                        act_suffix,
                        time_suffix,
                        max_value,
                        min_value,
                        eoc_index = int(3)):
    """
    Compute three performance metrics for suffix prediction:  
    - dl_distance: Normalized Damerau-Levenshtein distance between predicted and 
      ground truth activity label suffixes.
    - mae: Mean Absolute Error between predicted and ground truth TTNE suffixes.
    - msle: Mean Squared Logarithmic Error between predicted and ground truth 
      TTNE suffixes.
    
    Parameters
    ----------
    act_predictions : torch.Tensor
        Activity label suffix predictions.  
        Shape: (batch_size, suffix_len, num_act)
    time_predictions : torch.Tensor
        Timestamp suffix predictions.  
        Shape: (batch_size, suffix_len, 1)
    act_suffix : torch.Tensor
        Ground truth activity label suffix.  
        Shape: (batch_size, suffix_len)
    time_suffix : torch.Tensor
        Ground truth timestamp suffix.  
        Shape: (batch_size, suffix_len)
    max_value : float
        Maximum of the log-normalized TTNE.
    min_value : float
        Minimum of the log-normalized TTNE.
    eoc_index : int
        Index representing the End-Of-Case (EOC) token in the activity label 
        vocabulary. 
        Default is 3.
    
    Returns
    -------
    dl_distance : float
        Average normalized Damerau-Levenshtein distance over the batch.   
    mae : float
        Average MAE over the batch.
    msle : float
        Average MSLE over the batch.
    """
    # get batch_size
    batch_size = act_predictions.shape[0]

    # convert predicted activity label probabilities to lebel indices
    act_predictions = act_predictions.argmax(2) # shape: (batch_size, suffix_len)

    # reshape time_predictions
    time_predictions = time_predictions.squeeze(-1) # shape: (batch_size, suffix_len)

    # de-normalize and reverse log transformation of TTNE
    time_suffix = time_suffix * (max_value - min_value) + min_value
    time_suffix_exp = torch.exp(time_suffix) - 1   
    time_predictions = time_predictions * (max_value - min_value) + min_value
    time_predictions_exp = torch.exp(time_predictions) - 1

    # initialize performance metrics
    total_dl_distance, total_mae, total_msle = 0.0, 0.0, 0.0

    # define loss functions
    time_criterion_1 = nn.L1Loss()
    time_criterion_2 = nn.MSELoss()

    for i in range(batch_size):

        # -- normalized Damerau-Levenshtein distance --

        # convert predicted and target activity label sequences to lists, 
        # truncated at EOC
        act_pred_list = []
        act_target_list = []

        for p in act_predictions[i].tolist():
            act_pred_list.append(p)
            if p == eoc_index: # stop at EOC token (inclusive)
                break    
            
        for t in act_suffix[i].tolist():
            act_target_list.append(t)
            if t == eoc_index: # stop at EOC token (inclusive)
                break
        
        # compute normalized Damerau-Levenshtein distance
        pred_len, target_len= len(act_pred_list), len(act_target_list)
        max_len = max(pred_len, target_len)
        assert max_len > 0, "Error: max_len should be greater than 0."

        distance = damerau_levenshtein(act_pred_list, act_target_list)
        normalized_distance = distance / max_len
        total_dl_distance += normalized_distance

        # -- MAE --

        time_target_exp = time_suffix_exp[i, :target_len] # shape: (target_len,)
        time_pred_exp = time_predictions_exp[i, :target_len] # shape: (target_len,)
        mae_loss = time_criterion_1(time_target_exp, time_pred_exp)
        total_mae +=  mae_loss.item()

        # -- MSLE --

        time_target = time_suffix[i, :target_len] # shape: (target_len,)
        time_pred = time_predictions[i, :target_len] # shape: (target_len,)
        msle_loss = time_criterion_2(time_target, time_pred)
        total_msle +=  msle_loss.item() 

    # compute average metrics over the batch
    dl_distance = total_dl_distance / batch_size
    mae = total_mae / batch_size
    msle = total_msle / batch_size
    
    return dl_distance, mae, msle

In [ ]:
dl_distance, mae, msle = performance_metrics(act_predictions,
                                             time_predictions,
                                             test_suffix_act_tensor,
                                             test_suffix_time_tensor,
                                             max_pre,
                                             min_pre)
print(dl_distance)
print(mae)
print(msle)